In [1]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import census_key
c = Census(census_key, year=2018)

## Import Census Data 

In [2]:
#import census variable names csv
census_codes_csv = pd.read_csv('../Resources/census_columns.csv')
census_codes_csv.dropna(inplace = True)


#extract the columns of csv into lists
codes = [code for code in census_codes_csv['Code'] ]
names = [name for name in census_codes_csv['Name'] ]

In [3]:
#display the first four codes
codes[0:3]

['B01003_001E', 'B19301_001E', 'B01001A_001E']

In [4]:
#call census api
census_data = c.acs5.get(codes, {'for': 'zip code tabulation area:*'})

In [5]:
#create data frame
census_df = pd.DataFrame(census_data)
census_df

,B01003_001E,B19301_001E,B01001A_001E,B05003B_001E,B01001C_001E,B01001D_001E,B01001E_001E,B01001F_001E,B01001G_001E,B01001H_001E,B01001I_001E,zip code tabulation area
0,17242.0,6999.0,13026.0,145.0,25.0,3.0,0.0,3929.0,114.0,52.0,17184.0,00601
1,38442.0,9277.0,30529.0,1070.0,0.0,0.0,0.0,1540.0,5303.0,2102.0,35925.0,00602
2,48814.0,11307.0,37330.0,1930.0,115.0,364.0,10.0,8007.0,1058.0,698.0,47551.0,00603
3,6437.0,5943.0,2627.0,149.0,18.0,0.0,0.0,3518.0,125.0,10.0,6427.0,00606
4,27073.0,10220.0,20451.0,696.0,0.0,0.0,1.0,2871.0,3054.0,874.0,26043.0,00610
...,...,...,...,...,...,...,...,...,...,...,...,...
33115,76.0,NaN,76.0,0.0,0.0,0.0,0.0,0.0,0.0,76.0,0.0,63674
33116,10.0,NaN,10.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,95680
33117,33.0,NaN,33.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,0.0,30581
33118,6.0,NaN,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,97817


In [6]:
#create rename dictionary based on labels from the census columns csv
census_rename_dict = {codes[i]: names[i] for i in range(len(codes))} 
census_rename_dict['zip code tabulation area'] = 'zipcode'

#rename the columns
census_df.rename(columns = census_rename_dict,inplace = True)
census_df.head(2)

,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,other_race,two_or_more_races,white_not_hispanic,hispanic_or_latino,zipcode
0,17242.0,6999.0,13026.0,145.0,25.0,3.0,0.0,3929.0,114.0,52.0,17184.0,00601
1,38442.0,9277.0,30529.0,1070.0,0.0,0.0,0.0,1540.0,5303.0,2102.0,35925.0,00602


In [7]:
#drop na values
census_df.dropna(inplace = True)

#set zipcode as string for merge later
#chose string instead of int as some zipcodes start with 0
census_df['zipcode'] = census_df['zipcode'].astype('string')

## Extract zipcodes of interest

In [19]:
#list cities of interest
cities = ['philadelphia','atlanta','dallas','indianapolis','seattle']

#import csv files contiaining zipcodes for the cities
zipcodes = [pd.read_csv(f"../Resources/{city}_zipcodes.csv",encoding= 'unicode_escape') for city in cities ]

#drop the last row from every list as it is a reference link
zipcodes = [city.drop(city.tail(1).index) for city in zipcodes]

#make strings
zipcodes = [city.astype('string') for city in zipcodes]

#show dataframe for one of the cities
zipcodes[0].head()

,Philadelphia
0,19102
1,19103
2,19104
3,19106
4,19107


In [20]:
#aim to make one long dataframe with 2 columns: city and zipcode to later merge with the larger dataframe

#create dataframe with all codes
zipcodes_df = pd.concat(zipcodes)

#make one long list
zipcodes_df = zipcodes_df.stack()

#extract out the index and city for each zipcode
zipcodes_df = zipcodes_df.reset_index()

#drop the number index row
zipcodes_df.drop(columns = 'level_0', inplace = True)

#rename the columns
zipcodes_df.rename(columns = {'level_1' : 'city',0:'zipcode'},inplace = True)

#display
zipcodes_df

,city,zipcode
0,Philadelphia,19102
1,Philadelphia,19103
2,Philadelphia,19104
3,Philadelphia,19106
4,Philadelphia,19107
...,...,...
186,Seattle,98164
187,Seattle,98174
188,Seattle,98177
189,Seattle,98195


In [21]:
#merge dataframes so we have only the zipcodes we are interested in
census_cities_df = pd.merge(zipcodes_df,census_df,how = 'left', on = 'zipcode')

#display
census_cities_df

,city,zipcode,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,other_race,two_or_more_races,white_not_hispanic,hispanic_or_latino
0,Philadelphia,19102,4937.0,87197.0,3842.0,220.0,11.0,809.0,0.0,14.0,41.0,3583.0,259.0
1,Philadelphia,19103,24219.0,78815.0,18731.0,1499.0,72.0,2840.0,31.0,103.0,943.0,17468.0,1377.0
2,Philadelphia,19104,54311.0,15478.0,20419.0,22962.0,291.0,7662.0,62.0,838.0,2077.0,18782.0,2766.0
3,Philadelphia,19106,12375.0,82364.0,10179.0,1054.0,0.0,798.0,0.0,62.0,282.0,9495.0,797.0
4,Philadelphia,19107,13696.0,49131.0,8085.0,1117.0,73.0,3720.0,0.0,131.0,570.0,7600.0,703.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Seattle,98164,155.0,163222.0,123.0,0.0,0.0,32.0,0.0,0.0,0.0,123.0,0.0
187,Seattle,98174,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,Seattle,98177,21619.0,59544.0,17824.0,325.0,154.0,1735.0,0.0,196.0,1385.0,17167.0,882.0
189,Seattle,98195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning Data 

In [22]:
#create copy to clean
clean_df = census_cities_df.copy()

#find nan
clean_df[clean_df['population'].isnull()]

,city,zipcode,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,other_race,two_or_more_races,white_not_hispanic,hispanic_or_latino
139,Indianapolis,46183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,Seattle,98195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
clean_df.dropna(inplace=True)
clean_df

,city,zipcode,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,other_race,two_or_more_races,white_not_hispanic,hispanic_or_latino
0,Philadelphia,19102,4937.0,87197.0,3842.0,220.0,11.0,809.0,0.0,14.0,41.0,3583.0,259.0
1,Philadelphia,19103,24219.0,78815.0,18731.0,1499.0,72.0,2840.0,31.0,103.0,943.0,17468.0,1377.0
2,Philadelphia,19104,54311.0,15478.0,20419.0,22962.0,291.0,7662.0,62.0,838.0,2077.0,18782.0,2766.0
3,Philadelphia,19106,12375.0,82364.0,10179.0,1054.0,0.0,798.0,0.0,62.0,282.0,9495.0,797.0
4,Philadelphia,19107,13696.0,49131.0,8085.0,1117.0,73.0,3720.0,0.0,131.0,570.0,7600.0,703.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Seattle,98154,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186,Seattle,98164,155.0,163222.0,123.0,0.0,0.0,32.0,0.0,0.0,0.0,123.0,0.0
187,Seattle,98174,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,Seattle,98177,21619.0,59544.0,17824.0,325.0,154.0,1735.0,0.0,196.0,1385.0,17167.0,882.0


I notice that some seattle entries do not look right (population of zero, negative per capita income) I investigate below:

In [24]:
clean_df.agg(['count','mean','max','min'])

,city,zipcode,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,other_race,two_or_more_races,white_not_hispanic,hispanic_or_latino
count,189,189,189.000000,1.890000e+02,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000,189.000000
max,Seattle,98199,74971.000000,1.632220e+05,45201.000000,62340.000000,1386.000000,14652.000000,389.000000,14500.000000,3737.000000,41053.000000,39306.000000
min,Atlanta,19102,0.000000,-6.666667e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mean,NaN,inf,28276.037037,-2.465379e+07,15394.957672,8624.825397,115.470899,1806.174603,23.693122,1372.343915,938.571429,11824.169312,5295.529101


In [25]:
#investigate how many entries have a population of zero
clean_df[clean_df['population']<= 0]

,city,zipcode,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,other_race,two_or_more_races,white_not_hispanic,hispanic_or_latino
5,Philadelphia,19109,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Philadelphia,19112,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70,Atlanta,30334,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136,Dallas,75270,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,Dallas,75390,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,Seattle,98154,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
187,Seattle,98174,0.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
clean_df = clean_df[clean_df['population'] > 0 ]
clean_df

,city,zipcode,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,other_race,two_or_more_races,white_not_hispanic,hispanic_or_latino
0,Philadelphia,19102,4937.0,87197.0,3842.0,220.0,11.0,809.0,0.0,14.0,41.0,3583.0,259.0
1,Philadelphia,19103,24219.0,78815.0,18731.0,1499.0,72.0,2840.0,31.0,103.0,943.0,17468.0,1377.0
2,Philadelphia,19104,54311.0,15478.0,20419.0,22962.0,291.0,7662.0,62.0,838.0,2077.0,18782.0,2766.0
3,Philadelphia,19106,12375.0,82364.0,10179.0,1054.0,0.0,798.0,0.0,62.0,282.0,9495.0,797.0
4,Philadelphia,19107,13696.0,49131.0,8085.0,1117.0,73.0,3720.0,0.0,131.0,570.0,7600.0,703.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Seattle,98136,16607.0,60543.0,14120.0,622.0,55.0,729.0,13.0,253.0,815.0,13569.0,949.0
184,Seattle,98144,31845.0,49610.0,15333.0,5316.0,363.0,6341.0,49.0,2056.0,2387.0,13943.0,3772.0
186,Seattle,98164,155.0,163222.0,123.0,0.0,0.0,32.0,0.0,0.0,0.0,123.0,0.0
188,Seattle,98177,21619.0,59544.0,17824.0,325.0,154.0,1735.0,0.0,196.0,1385.0,17167.0,882.0


In [27]:
clean_df.loc[:,'population':].agg(['mean','median','std','max','min'])

,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,other_race,two_or_more_races,white_not_hispanic,hispanic_or_latino
mean,29363.576923,39010.752747,15987.071429,8956.549451,119.912088,1875.642857,24.604396,1425.126374,974.670330,12278.945055,5499.203297
median,28145.500000,31334.500000,14841.500000,4074.000000,74.000000,946.000000,0.000000,567.500000,809.000000,10893.500000,2273.000000
std,16235.874651,24608.927443,11042.207136,11094.581106,158.391092,2436.966722,56.760402,2637.443026,777.568175,9375.433742,7694.935461
max,74971.000000,163222.000000,45201.000000,62340.000000,1386.000000,14652.000000,389.000000,14500.000000,3737.000000,41053.000000,39306.000000
min,155.000000,2465.000000,33.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000


In [28]:
clean_df

,city,zipcode,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,other_race,two_or_more_races,white_not_hispanic,hispanic_or_latino
0,Philadelphia,19102,4937.0,87197.0,3842.0,220.0,11.0,809.0,0.0,14.0,41.0,3583.0,259.0
1,Philadelphia,19103,24219.0,78815.0,18731.0,1499.0,72.0,2840.0,31.0,103.0,943.0,17468.0,1377.0
2,Philadelphia,19104,54311.0,15478.0,20419.0,22962.0,291.0,7662.0,62.0,838.0,2077.0,18782.0,2766.0
3,Philadelphia,19106,12375.0,82364.0,10179.0,1054.0,0.0,798.0,0.0,62.0,282.0,9495.0,797.0
4,Philadelphia,19107,13696.0,49131.0,8085.0,1117.0,73.0,3720.0,0.0,131.0,570.0,7600.0,703.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,Seattle,98136,16607.0,60543.0,14120.0,622.0,55.0,729.0,13.0,253.0,815.0,13569.0,949.0
184,Seattle,98144,31845.0,49610.0,15333.0,5316.0,363.0,6341.0,49.0,2056.0,2387.0,13943.0,3772.0
186,Seattle,98164,155.0,163222.0,123.0,0.0,0.0,32.0,0.0,0.0,0.0,123.0,0.0
188,Seattle,98177,21619.0,59544.0,17824.0,325.0,154.0,1735.0,0.0,196.0,1385.0,17167.0,882.0


In [29]:
#export to csv
clean_df.to_csv('../Output/Data/census_data.csv', header = True, index = False)